# A standardized and reproducible method to measure decision-making in mice: Data

This Jupyter notebooks shows how to access the data via the [Open Neurophysiology Environment (ONE) interface in Python](https://ibllib.readthedocs.io/en/latest/02_tutorial_python.html). The particular ONE implementation used in this notebook ("ONE light") only requires a local copy of the data, organized in a hierarchy of appropriately named subfolders and file names.

We start by importing the ONE light implementation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn

from oneibl.onelight import ONE

By default, this ONE implementation looks for data in the current directory.

In [ ]:
pwd

In [ ]:
!cd one_data/ibl-behavioral-data-Dec2019

We create the main ONE() instance that we will use to access the data.

In [ ]:
one = ONE()

We search all sessions that have a given dataset type pattern.

In [ ]:
eids = one.search(['_ibl_trials.*'])

We select the tenth session.

In [ ]:
eid = eids[10]
print(f"Loading session {eid}.")

We list all dataset types available in that session.

In [ ]:
dset_types = one.list(eid)
print(f"Available dataset types: {', '.join(dset_types)}")

We load a single dataset.

In [ ]:
print(f"Loading {dset_types[0]}")
choice = one.load_dataset(eid, dset_types[0])
print(choice.shape)

We load an entire object (a kind of Python dictionary).

In [ ]:
print("Loading the _ibl_trials object.")
trials = one.load_object(eid, "_ibl_trials")
for key, value in trials.items():
    print(key, value.shape)

We make a very basic plot.

In [ ]:
out = []
for sgn, contrast in ((-1, trials.contrastRight), (+1, trials.contrastLeft)):
    for c in np.unique(contrast)[::sgn]:
        if not np.isnan(c) and (c != 0 or sgn == +1):
            out.append((sgn * c, (trials.choice[contrast == c] == +1).mean()))
out = np.array(out) * 100

plt.figure(figsize=(10, 6))
plt.plot(out[:, 0], out[:, 1], 'o-', lw=4, ms=10)
plt.xlabel("Signed contrast (%)")
plt.ylabel("Rightward choice (%)")
plt.ylim(0, 100)
plt.title("Psychometric curve for %s" % eid);